In [ ]:

#visualization tool for displaying long load/processing times
!pip install tqdm --quiet
#data processing
!pip install pandas --quiet
#workhorse for converting text into embeddings/vectors
!pip install sentence-transformers==2.2.2 --quiet
#data framework for LLM applications
!pip install llama-index==0.9.29  --quiet
#logging output
!pip install loguru==0.7.0 --quiet
#convenient pretty printing library
!pip install rich --quiet
#openai Tokenizer library
!pip install tiktoken --quiet

In [2]:
!curl -o Split_Data_Med.json https://raw.githubusercontent.com/paramjeetn/Doctor_Copilot_V1/main/Data/Split_Data_Med.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  124k  100  124k    0     0   776k      0 --:--:-- --:--:-- --:--:--  780k


In [ ]:
!pip install chromadb --quiet

In [4]:
%load_ext autoreload
%autoreload 2

#standard libraries
import sys
sys.path.append('../')

import json
import os
import time
from typing import List, Tuple
from math import ceil

#external libraries
import pandas as pd
import numpy as np
from rich import print
from rich.pretty import pprint #nifty library for pretty printing
from torch import cuda
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
root_folder = '/content/'  #'../data'
data_file = 'Split_Data_Med.json'
data_path = os.path.join(root_folder, data_file)


def load_json(file_path: str) -> list[dict]:
  with open(file_path) as f:
    data = json.load(f)
  return data

data = load_json(data_path)
# print(f'Total # of papers: {len(data)}')
# print(data[:1])

In [7]:
import json
import chromadb
#change path accordingly for local and colab
path = '/content/gdrive/MyDrive/Data_Medical_Rag/ChromaDb'
client = chromadb.PersistentClient(path)

In [8]:
client.heartbeat()

1719763239891586105

In [9]:
collection = client.get_or_create_collection("Doctor_Copilot_chromaDB_Collection")


In [10]:
data = load_json(data_path)

documents_embeddings = []
metadata_list = []
ids_list = []

for doc in data:
    # Assuming you need to convert the content into a list of embeddings
    # Replace this with your actual embedding conversion logic
    #adding summary

    #adding content
    documents_embeddings.append(doc["content"])  # Split the content string into floats
    metadata_list.append({"doc_id": doc["doc_id"], "doc_name": doc["doc_name"]})
    ids_list.append(doc["content_id"])

# print("Adding to collection:")
# print("Documents:", documents_embeddings)
# print("Metadata:", metadata_list)
# print("IDs:", ids_list)

collection.upsert(
    documents=documents_embeddings,
    ids=ids_list
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 45.2MiB/s]


In [15]:

collection.query(
    query_texts=["What are the specific recommendations for screening and treatment of nonalcoholic fatty liver disease (NAFLD) in people with type 2 diabetes who are over 50 years old?"],
    n_results=2,
)


{'ids': [['d_8_cont_3', 'd_8_cont_5']],
 'distances': [[0.5174748301506042, 0.7153717279434204]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['Consider antiresorptive and osteo-anabolic agents for those with aT-score ≤−2.0 or previous fragility fractures. Question: Are people with diabetes at increased risk for cancer? Ans: Diabetes is associated with increased risk of cancers of the liver, pancreas, endometrium, colon/rectum, breast, and bladder. Nevertheless, cancer screening recommendations are the same for people with diabetes as for those without diabetes. Question: How prevalent is nonalcoholic fatty liver disease (NAFLD)? Who should be screened for it and how? Ans: i) Prevalence Approximately 70% of people with type 2 diabetes have NAFLD.',
   'Indeterminate Risk: Further Testing: Proceed with vibration-controlled transient elastography or an ELF (Enhanced Liver Fibrosis) blood test to refine the risk assessment. After further testing: Low Risk: Continue t

In [ ]:
from google.colab import files

folder_path = '/content/gdrive/MyDrive/Data_Medical_Rag/ChromaDb'
!zip -r {folder_path}.zip {folder_path}
files.download(f'{folder_path}.zip')